# 04. Merging Datasets

Merging is one of the most powerful skills in data analysis. You'll use it constantly to combine information from different sources, like adding stroke counts to your IDS table, or finding which characters appear in multiple variant lists.

You can do glorious things merging data. First, however, you will be merging things that don't work until 4 A.M.

Let's load the IDS table and the clean stroke count table that Marina so kindly cleaned for us in the last notebook:


In [ ]:
import pandas as pd

# Load the IDS table
ids_df = pd.read_csv('../daniel_tables/ids_df.csv',
    index_col=None,
    encoding='utf-8')

# Load the clean stroke count table
stroke_df = pd.read_csv('../marina_tables/stroke_count_clean.csv',
    index_col=None,
    encoding='utf-8')

# Print the head of both tables
print("IDS Table:")
print(ids_df.head())
print(f"\nIDS Table shape: {ids_df.shape}")
print("\n" + "="*50 + "\n")
print("Stroke Count Table:")
print(stroke_df.head())
print(f"\nStroke Count Table shape: {stroke_df.shape}")


OK, so we have two tables. The IDS table has character decomposition information, and the stroke count table has... well, stroke counts. Wouldn't it be nice if we could combine them? Like, have one table with both the components AND the stroke count for each character?

That's what merging is for! But here's the thing: not every character in the IDS table has a stroke count, and not every character in the stroke count table has IDS decomposition data. So we need to decide: what do we want to keep?

Pandas gives us several options, and they're called "joins". Let's see what happens with each one:


## Inner Join: Only Matching Characters

An inner join keeps ONLY the rows where the character exists in BOTH tables. This is the most restrictive option - if a character doesn't have a stroke count, it gets dropped. If a character doesn't have IDS data, it gets dropped.

Use this when you only want complete data - characters that have information in both tables.


In [ ]:
# Inner join: only keep characters that exist in BOTH tables
df_inner = pd.merge(ids_df, stroke_df,
                    on='character',
                    how='inner')

print(f"Original IDS table: {len(ids_df)} rows")
print(f"Original stroke count table: {len(stroke_df)} rows")
print(f"After inner join: {len(df_inner)} rows")
print(f"\nLost {len(ids_df) - len(df_inner)} rows from IDS table")
print(f"Lost {len(stroke_df) - len(df_inner)} rows from stroke count table")
print("\nFirst few rows:")
print(df_inner.head())


Simplified → Traditional mappings:
Rows: 8399

Traditional → Simplified mappings:
Rows: 5097


,traditional,simplified
8588,㐷,傌
8589,㐹,㑶
8590,㐽,偑
8591,㑇,㑳
8592,㑈,倲


## Left Join: Keep All IDS Characters

A left join keeps ALL rows from the LEFT table (ids_df) and adds matching data from the RIGHT table (stroke_df) where it exists. If there's no match, the stroke count columns will be empty (NaN).

Use this when you want to keep all your IDS data, even if some characters don't have stroke counts yet.


In [ ]:
# Left join: keep all IDS characters, add stroke counts where available
df_left = pd.merge(ids_df, stroke_df,
                   on='character',
                   how='left')

print(f"Original IDS table: {len(ids_df)} rows")
print(f"After left join: {len(df_left)} rows")
print(f"\nKept all {len(ids_df)} rows from IDS table!")
print(f"Added stroke counts for {df_left['stroke_count'].notna().sum()} characters")
print(f"Missing stroke counts for {df_left['stroke_count'].isna().sum()} characters")
print("\nFirst few rows:")
print(df_left.head())
print("\nSome rows with missing stroke counts:")
print(df_left[df_left['stroke_count'].isna()].head())


## Right Join: Keep All Stroke Count Characters

A right join keeps ALL rows from the RIGHT table (stroke_df) and adds matching data from the LEFT table (ids_df) where it exists. If there's no match, the IDS columns will be empty (NaN).

Use this when you want to keep all your stroke count data, even if some characters don't have IDS decomposition yet.


In [ ]:
# Right join: keep all stroke count characters, add IDS data where available
df_right = pd.merge(ids_df, stroke_df,
                    on='character',
                    how='right')

print(f"Original stroke count table: {len(stroke_df)} rows")
print(f"After right join: {len(df_right)} rows")
print(f"\nKept all {len(stroke_df)} rows from stroke count table!")
print(f"Added IDS data for {df_right['components'].notna().sum()} characters")
print(f"Missing IDS data for {df_right['components'].isna().sum()} characters")
print("\nFirst few rows:")
print(df_right.head())
print("\nSome rows with missing IDS data:")
print(df_right[df_right['components'].isna()].head())


## Comparing the Results

OK Marina, let's see what we learned. Why are the sizes different?

The key is understanding what each join type keeps:


In [ ]:
# Compare the sizes
print("Summary of merge results:")
print(f"{'='*60}")
print(f"Original IDS table:           {len(ids_df):,} rows")
print(f"Original stroke count table:   {len(stroke_df):,} rows")
print(f"{'='*60}")
print(f"Inner join:  {len(df_inner):,} rows (only characters in BOTH tables)")
print(f"Left join:   {len(df_left):,} rows (all IDS + matching stroke counts)")
print(f"Right join:  {len(df_right):,} rows (all stroke counts + matching IDS)")
print(f"{'='*60}")
print(f"\nWhy is inner join smaller?")
print(f"  → Because {len(ids_df) - len(df_inner):,} characters in IDS table don't have stroke counts")
print(f"  → And {len(stroke_df) - len(df_inner):,} characters in stroke count table don't have IDS data")
print(f"\nWhy are left and right joins different sizes?")
print(f"  → Left join keeps all {len(ids_df):,} IDS characters (some without stroke counts)")
print(f"  → Right join keeps all {len(stroke_df):,} stroke count characters (some without IDS data)")
print(f"  → They're different because the two tables have different characters!")


Perfect! Now you understand the difference. For most of your work, you'll probably want to use **left join** to keep all your IDS data and add stroke counts where available.

But wait, there's more! Merging isn't just for combining different tables - you can also merge a table **onto itself**. This is super useful for finding relationships within the same data.

Let's do something cool: we're going to extract all the **radical components** from each character's decomposition. Not just the direct components, but the components of those components, and the components of THOSE components. We'll go 2-3 levels deep, and then you can go deeper on your own time if you want.

First, let's define what we mean by "radicals":


In [ ]:
# Standard radicals list (from Jisho.org)
radicals = [
    '一', '｜', '丶', 'ノ', '乙', '亅', '二', '亠', '人', '⺅', '𠆢', '儿', '入', 'ハ', '丷', '冂', 
    '冖', '冫', '几', '凵', '刀', '⺉', '力', '勹', '匕', '匚', '十', '卜', '卩', '厂', '厶', '又', 
    'マ', '九', 'ユ', '乃', '𠂉', '⻌', '口', '囗', '土', '士', '夂', '夕', '大', '女', '子', '宀', 
    '寸', '小', '⺌', '尢', '尸', '屮', '山', '川', '巛', '工', '已', '巾', '干', '幺', '广', '廴', 
    '廾', '弋', '弓', 'ヨ', '彑', '彡', '彳', '⺖', '⺘', '⺡', '⺨', '⺾', '⻏⻖', '也', '亡', '及', 
    '久', '⺹', '心', '戈', '戸', '手', '支', '攵', '文', '斗', '斤', '方', '无', '日', '曰', '月', 
    '木', '欠', '止', '歹', '殳', '比', '毛', '氏', '气', '水', '火', '⺣', '爪', '父', '爻', '爿', 
    '片', '牛', '犬', '⺭', '王', '元', '井', '勿', '尤', '五', '屯', '巴', '毋', '玄', '瓦', '甘', 
    '生', '用', '田', '疋', '疒', '癶', '白', '皮', '皿', '目', '矛', '矢', '石', '示', '禸', '禾', 
    '穴', '立', '⻂', '世', '巨', '冊', '母', '⺲', '牙', '瓜', '竹', '米', '糸', '缶', '羊', '羽', 
    '而', '耒', '耳', '聿', '肉', '自', '至', '臼', '舌', '舟', '艮', '色', '虍', '虫', '血', '行', 
    '衣', '西', '臣', '見', '角', '言', '谷', '豆', '豕', '豸', '貝', '赤', '走', '足', '身', '車', 
    '辛', '辰', '酉', '釆', '里', '舛', '麦', '金', '長', '門', '隶', '隹', '雨', '青', '非', '奄', 
    '岡', '免', '斉', '面', '革', '韭', '音', '頁', '風', '飛', '食', '首', '香', '品', '馬', '骨', 
    '高', '髟', '鬥', '鬯', '鬲', '鬼', '竜', '韋', '魚', '鳥', '鹵', '鹿', '麻', '亀', '啇', '黄', 
    '黒', '黍', '黹', '無', '歯', '黽', '鼎', '鼓', '鼠', '鼻', '齊', '龠'
]

print(f"Total radicals: {len(radicals)}")
print(f"First 20: {radicals[:20]}")


## Level 1: Extract Direct Radical Components

First, we need to extract individual characters from the `components` column. The components column contains IDS notation with structure characters like ⿰⿱⿲⿳⿴⿵⿶⿷⿸⿹⿺⿻ that tell us how components are arranged. We want to extract just the actual CJK characters and filter for radicals.

Here's a function to extract characters from an IDS string:


In [ ]:
# Function to extract CJK characters from IDS notation
def extract_characters(ids_string):
    """Extract individual CJK characters from an IDS string"""
    if pd.isna(ids_string):
        return []
    
    # IDS structure characters that we want to remove
    structure_chars = '⿰⿱⿲⿳⿴⿵⿶⿷⿸⿹⿺⿻'
    
    # Extract only CJK characters (Unicode range roughly 0x4E00-0x9FFF for CJK Unified Ideographs)
    # Also include some extended ranges for radicals and variants
    chars = []
    for char in str(ids_string):
        # Skip structure characters and arrows (→, ←)
        if char in structure_chars or char in '→←':
            continue
        # Check if it's a CJK character (rough check)
        code = ord(char)
        if (0x4E00 <= code <= 0x9FFF) or (0x3400 <= code <= 0x4DBF) or (0x20000 <= code <= 0x2A6DF):
            chars.append(char)
    
    return chars

# Test the function
test_string = "⿰歲刂"
print(f"Test: {test_string} → {extract_characters(test_string)}")

# Apply to the components column
ids_df['components_list'] = ids_df['components'].apply(extract_characters)

# Extract only radicals from Level 1 (direct components)
ids_df['radicals_level1'] = ids_df['components_list'].apply(
    lambda chars: [c for c in chars if c in radicals]
)

print("\nFirst few rows with extracted radicals:")
print(ids_df[['character', 'components', 'radicals_level1']].head(10))


Good! Now we have Level 1 radicals - the radicals that appear directly in each character's components. But what about characters that are made up of other characters, which are in turn made up of radicals? We need to go deeper!

## Level 2: Components of Components

Now we're going to merge the table onto itself. For each character in the components list that isn't a radical, we'll look up its components and extract radicals from those.

This is where self-joining gets interesting:


In [ ]:
# Step 1: Create a lookup table of character -> components
# We'll use this to look up components of components
component_lookup = ids_df[['character', 'components_list']].copy()

# Step 2: For each row, get all non-radical components
# Then look up their components
def get_level2_radicals(row):
    """Get radicals from Level 2 (components of components)"""
    level2_radicals = []
    
    # Get all components from Level 1 that aren't radicals
    non_radical_components = [c for c in row['components_list'] if c not in radicals]
    
    # For each non-radical component, look up its components
    for comp in non_radical_components:
        # Find this component in our lookup table
        comp_data = component_lookup[component_lookup['character'] == comp]
        if not comp_data.empty:
            comp_components = comp_data.iloc[0]['components_list']
            # Extract radicals from these components
            comp_radicals = [c for c in comp_components if c in radicals]
            level2_radicals.extend(comp_radicals)
    
    return level2_radicals

# Apply the function
ids_df['radicals_level2'] = ids_df.apply(get_level2_radicals, axis=1)

print("First few rows with Level 2 radicals:")
print(ids_df[['character', 'components', 'radicals_level1', 'radicals_level2']].head(10))


Excellent! Now we're getting radicals from components of components. But wait, there's more! What if those components-of-components also have components? Let's go one level deeper:


In [ ]:
# Level 3: Components of components of components
def get_level3_radicals(row):
    """Get radicals from Level 3 (components of components of components)"""
    level3_radicals = []
    
    # Get all non-radical components from Level 1
    non_radical_components = [c for c in row['components_list'] if c not in radicals]
    
    # For each non-radical component, get its components
    for comp in non_radical_components:
        comp_data = component_lookup[component_lookup['character'] == comp]
        if not comp_data.empty:
            comp_components = comp_data.iloc[0]['components_list']
            
            # Now get components of these components (Level 3)
            for comp2 in comp_components:
                if comp2 not in radicals:  # Only look up if it's not already a radical
                    comp2_data = component_lookup[component_lookup['character'] == comp2]
                    if not comp2_data.empty:
                        comp2_components = comp2_data.iloc[0]['components_list']
                        # Extract radicals from Level 3
                        comp2_radicals = [c for c in comp2_components if c in radicals]
                        level3_radicals.extend(comp2_radicals)
    
    return level3_radicals

# Apply the function
ids_df['radicals_level3'] = ids_df.apply(get_level3_radicals, axis=1)

print("First few rows with all three levels:")
print(ids_df[['character', 'components', 'radicals_level1', 'radicals_level2', 'radicals_level3']].head(10))


Perfect! Now we have radicals from three levels. The final step is to combine them all into a single column with unique radicals (no duplicates):


In [ ]:
# Combine all levels into a single list of unique radicals
def combine_all_radicals(row):
    """Combine radicals from all levels, removing duplicates"""
    all_radicals = []
    all_radicals.extend(row['radicals_level1'])
    all_radicals.extend(row['radicals_level2'])
    all_radicals.extend(row['radicals_level3'])
    
    # Remove duplicates while preserving order
    seen = set()
    unique_radicals = []
    for rad in all_radicals:
        if rad not in seen:
            seen.add(rad)
            unique_radicals.append(rad)
    
    return unique_radicals

ids_df['radical_components'] = ids_df.apply(combine_all_radicals, axis=1)

print("Final result with combined radical_components column:")
print(ids_df[['character', 'components', 'radical_components']].head(10))

print("\nSome examples with multiple radicals:")
examples = ids_df[ids_df['radical_components'].apply(len) > 2].head(5)
for idx, row in examples.iterrows():
    print(f"\n{row['character']}: {row['components']}")
    print(f"  Radicals: {row['radical_components']}")


Awesome! Now you have a column that lists all the radical components for each character, going 2-3 levels deep. 

## What We Learned

1. **Inner Join**: Only keeps rows where the key exists in BOTH tables
2. **Left Join**: Keeps all rows from the LEFT table, adds matching data from RIGHT
3. **Right Join**: Keeps all rows from the RIGHT table, adds matching data from LEFT
4. **Self-Join**: Merge a table onto itself to find relationships within the same data

## Summary

| Merge Type | `how=` | Keeps Rows From | Use When |
|------------|--------|------------------|----------|
| Inner | `'inner'` | Both DataFrames | You only want matching records |
| Left | `'left'` | Left DataFrame | Keep all from left, add matching from right |
| Right | `'right'` | Right DataFrame | Keep all from right, add matching from left |

## Key Parameters

- `on='column'` - Merge on a column with the same name in both DataFrames
- `left_on='col1', right_on='col2'` - Merge on columns with different names
- `how='inner'` - Type of join (inner, left, right, outer)

## Try It Yourself

1. Experiment with going deeper (Level 4, Level 5) to see how many radicals you can extract
2. Try merging the stroke count data with the final radical_components column
3. Count how many characters have each radical
4. Find characters that share the same set of radicals

## What's Next?

In the next notebook, we'll learn how to:
- Transform and manipulate data
- Create new columns
- Build lookup tables for dictionaries and character composers
